Cada línea del archivo movies.csv representa a un actor que actuó en una película. Si una película tiene diez actores, habrá diez filas para esa película en particular.



1. Calcule la cantidad de películas en las que participó cada actor. La salida debe tener dos columnas: actor y conteo. La salida debe ordenarse por el conteo en orden descendente.

2. Calcule la cantidad de películas producidas cada año. La salida debe tener tres columnas: año, siglo al que pertenece el año y conteo. La salida debe ordenarse por el conteo en orden descendente.

3. Obtenga la película con la calificación más alta por año. La salida debe tener solo una película por año y debe contener tres columnas: año, título de la película y valoración.

In [40]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import col, count, desc, udf, rank, countDistinct
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window

spark = SparkSession.builder.master("local[*]").getOrCreate()

sc: SparkContext = spark.sparkContext

In [4]:
movies_df = spark.read.csv("./data/s9_in/movies.csv", header=True, inferSchema=True, sep="|")
movie_ratings_df = spark.read.csv("./data/s9_in/movie_ratings.csv", header=True, inferSchema=True, sep="|")

movies_df.printSchema()
movie_ratings_df.printSchema()

movies_df.show(5, truncate=False)
movie_ratings_df.show(5, truncate=False)

root
 |-- actor: string (nullable = true)
 |-- pelicula: string (nullable = true)
 |-- año: integer (nullable = true)

root
 |-- valoracion: double (nullable = true)
 |-- pelicula: string (nullable = true)
 |-- año: integer (nullable = true)

+-----------------+-------------+----+
|actor            |pelicula     |año |
+-----------------+-------------+----+
|McClure, Marc (I)|Freaky Friday|2003|
|McClure, Marc (I)|Coach Carter |2005|
|McClure, Marc (I)|Superman II  |1980|
|McClure, Marc (I)|Apollo 13    |1995|
|McClure, Marc (I)|Superman     |1978|
+-----------------+-------------+----+
only showing top 5 rows

+----------+--------------------------+----+
|valoracion|pelicula                  |año |
+----------+--------------------------+----+
|1.6339    |'Crocodile' Dundee II     |1988|
|7.6177    |10                        |1979|
|1.2864    |10 Things I Hate About You|1999|
|0.3243    |10,000 BC                 |2008|
|0.3376    |101 Dalmatians            |1996|
+----------+---------

In [41]:
# 1. Calcule la cantidad de películas en las que participó cada actor. La salida debe tener dos columnas: actor y conteo. La salida debe ordenarse por el conteo en orden descendente.

films_by_actor_df = (
    movies_df
    .groupBy(
        "actor"
    )
    .agg(
        count("pelicula").alias("n_films")
    )
)

films_by_actor_df.orderBy(desc("n_films")).show(
    n=films_by_actor_df.count(),
    truncate=False,
)

+----------------------------+-------+
|actor                       |n_films|
+----------------------------+-------+
|Welker, Frank               |38     |
|Tatasciore, Fred            |38     |
|Jackson, Samuel L.          |32     |
|Harnell, Jess               |31     |
|Damon, Matt                 |27     |
|Willis, Bruce               |27     |
|Cummings, Jim (I)           |26     |
|Lynn, Sherry (I)            |25     |
|McGowan, Mickie             |25     |
|Bergen, Bob (I)             |25     |
|Hanks, Tom                  |25     |
|Proctor, Phil               |24     |
|Pitt, Brad                  |23     |
|Wilson, Owen (I)            |23     |
|Cruise, Tom                 |23     |
|Depp, Johnny                |22     |
|Morrison, Rana              |22     |
|Freeman, Morgan (I)         |22     |
|Williams, Robin (I)         |22     |
|Buscemi, Steve              |21     |
|Mann, Danny (I)             |21     |
|Neeson, Liam                |21     |
|Murphy, Eddie (I)       

In [43]:
# 2. Calcule la cantidad de películas producidas cada año. La salida debe tener tres columnas: año, siglo al que pertenece el año y conteo. La salida debe ordenarse por el conteo en orden descendente.

@udf(returnType=IntegerType())
def century(year: int) -> int:
    return year // 100 + 1

films_by_year = (
    movies_df
    .groupBy("año")
    .agg(
        countDistinct("pelicula").alias("total_films")
    )
    .withColumn(
        "century",
        century(col("año")),
    )
    .orderBy(
        desc("total_films"),
    )
)

films_by_year.show(
    n=films_by_year.count(),
    truncate=False,
)

+----+-----------+-------+
|año |total_films|century|
+----+-----------+-------+
|2011|86         |21     |
|2004|86         |21     |
|2006|86         |21     |
|2005|85         |21     |
|2008|82         |21     |
|2002|81         |21     |
|2010|78         |21     |
|2000|77         |21     |
|2003|76         |21     |
|2007|75         |21     |
|2001|71         |21     |
|2009|68         |21     |
|1999|67         |20     |
|1997|66         |20     |
|1998|59         |20     |
|1996|42         |20     |
|2012|32         |21     |
|1995|25         |20     |
|1994|16         |20     |
|1986|16         |20     |
|1992|14         |20     |
|1984|13         |20     |
|1993|13         |20     |
|1990|11         |20     |
|1985|11         |20     |
|1987|9          |20     |
|1988|9          |20     |
|1982|9          |20     |
|1989|8          |20     |
|1983|8          |20     |
|1991|8          |20     |
|1981|6          |20     |
|1979|5          |20     |
|1977|3          |20     |
|

In [45]:
# 3. Obtenga la película con la calificación más alta por año. La salida debe tener solo una película por año y debe contener tres columnas: año, título de la película y valoración.

window_spec = Window().partitionBy(movie_ratings_df.año).orderBy(desc(movie_ratings_df.valoracion))

foty = (
    movie_ratings_df
    .withColumn(
        "rank",
        rank().over(window_spec)
    )
    .filter(
        col("rank") == 1,
    )
    .select(
        "año",
        "pelicula",
        "valoracion",
        "rank",
    )
    .orderBy(
        col("año").desc(),
    )
)

foty.show(
    n=foty.count(),
    truncate=False,
)



+----+-------------------------------+----------+----+
|año |pelicula                       |valoracion|rank|
+----+-------------------------------+----------+----+
|2013|The Wolverine                  |12.5      |1   |
|2012|1066                           |12.8205   |1   |
|2011|Ang babae sa septic tank       |14.1527   |1   |
|2010|Beginners                      |14.2173   |1   |
|2009|Kimmy Dora: Kambal sa kiyeme   |13.7234   |1   |
|2008|Man on Wire                    |14.0356   |1   |
|2007|Hostel: Part II                |13.7432   |1   |
|2006|Love and Other Disasters       |13.7696   |1   |
|2005|The Man                        |14.1976   |1   |
|2004|Sleepover                      |14.2073   |1   |
|2003|Gigli                          |14.1829   |1   |
|2002|Extreme Ops                    |12.8821   |1   |
|2001|Tortilla Soup                  |14.0009   |1   |
|2000|Taxi 2                         |14.1178   |1   |
|1999|Sofies verden                  |14.0421   |1   |
|1998|My G